# 调试和显示 query 全过程

In [2]:
%%time

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 42 µs, sys: 0 ns, total: 42 µs
Wall time: 44.3 µs


In [3]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 3.3 s, sys: 414 ms, total: 3.71 s
Wall time: 3.37 s


In [4]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    result = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    result['url']=url
    return result
    

CPU times: user 6.45 ms, sys: 3.58 ms, total: 10 ms
Wall time: 9.46 ms


In [5]:
get_news_data("https://www.guancha.cn/internation/2024_06_20_738635.shtml")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
DEBUG:urllib3.connectionpool:https://www.guancha.cn:443 "GET /internation/2024_06_20_738635.shtml HTTP/1.1" 200 15661
https://www.guancha.cn:443 "GET /internation/2024_06_20_738635.shtml HTTP/1.1" 200 15661
https://www.guancha.cn:443 "GET /internation/2024_06_20_738635.shtml HTTP/1.1" 200 15661


{'title': '沙特麦加酷热难耐，“已有来自埃及、约旦等国超900名朝觐者死亡”',
 'author': '齐倩',
 'publish_time': '2024-06-20 15:24:32',
 'content': '【文/观察者网 齐倩】6月，沙特进入一年中最热月份，高温导致正在“圣城”麦加参加朝觐活动的大批朝觐者死亡。由于官方没有公布具体数据，死亡人数尚不确定。法新社最新统计称，截至当地时间6月19日，死亡人数已飙升至900多人。\n朝觐是伊斯兰教的五大基本功课之一，所有身体健全且有经济能力的穆斯林在一生之中都须完成至少一次朝觐。据伊斯兰历法，朝觐季节每年都会变化，今年是在六月，也是沙特最热的月份之一。\n沙特官方数据显示，来自世界各地的约180万穆斯林参加了为期数天的朝觐活动，其中约160万人来自国外，不乏年老体弱的人。据介绍，每年的朝觐活动都会吸引数十万来自低收入国家的朝觐者，“其中许多人几乎没有接受过朝觐前的医疗保健”，导致传染病传播。\n另外，每年都有数以万计的朝觐者为了省钱，试图在没有获得官方签证的情况下参加朝觐。沙特近期的酷热高温，给朝觐活动又增添危险：非法朝觐者无法使用朝觐路线上提供的空调设施等降暑服务。\n当地时间17日，麦加气温达到51.8摄氏度。与此同时，沙特以及其他国家的民众纷纷前往医院，或者在社交媒体焦急地发帖求助，试图寻找朝觐期间失踪的亲友。\n麦加气温超过50摄氏度，大批朝觐者死于高温（图自视觉中国，下同）\n沙特没有提供死亡人数的具体信息，但仅在上周日（16日）就报告了2700多例“中暑”病例。由于缺乏官方数据，死亡人数尚未确定，且可能会继续上升。\n美联社19日报道称，网上流传的一份名单显示，数日内已有至少550人死亡。法新社则对各国公布数据统计得出，截至当地时间19日，死亡人数已飙升至900多人。一名阿拉伯地区的外交官告诉法新社，仅埃及人的死亡人数就从18日的300多人跃升至19日的“至少600人”，其中大部分死于高温。\n还有一名外交官透露，除了一人在小规模人群挤压中受重伤致死外，“所有埃及人均因高温死亡”。他还表示，死亡总数来自麦加当地医院太平间的数据。\n当地时间18日，埃及外交部曾发布声明表示，埃及正在与沙特官方合作，搜寻在朝觐期间失踪的埃及人。除埃及外，约旦、印度尼西亚、印度、伊朗、塞内加尔、突尼斯和伊拉

In [6]:
%%time

!rm -rf ./index

CPU times: user 5.49 ms, sys: 7.49 ms, total: 13 ms
Wall time: 122 ms


In [7]:
%%time

import os
from llama_index.core import(
    StorageContext, 
    load_index_from_storage,
    get_response_synthesizer,
    DocumentSummaryIndex,
    Document
)
from llama_index.core.node_parser import SentenceSplitter

def _load_index(folder_path):
    # 考虑不存在的情况，创建空的索引
    if not os.path.exists(folder_path):
        splitter = SentenceSplitter(chunk_size=1024)
        response_synthesizer = get_response_synthesizer(
            response_mode="tree_summarize", use_async=True,
        )
        
        doc_summary_index = DocumentSummaryIndex.from_documents(
            [],
            transformations=[splitter],
            response_synthesizer=response_synthesizer,
            show_progress=True,
        )
        
        doc_summary_index.storage_context.persist(folder_path)
        
    storage_context = StorageContext.from_defaults(persist_dir=folder_path)
    return load_index_from_storage(storage_context)

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 22.4 µs


In [8]:
%%time

doc_summary_index=_load_index('index')

Parsing nodes: 0it [00:00, ?it/s]

Summarizing documents: 0it [00:00, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

DEBUG:fsspec.local:open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/docstore.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/docstore.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/docstore.json
DEBUG:fsspec.local:open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/index_store.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/index_store.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/index_store.json
DEBUG:fsspec.local:open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/graph_store.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/graph_store.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/graph_store.json
DEBUG:fsspec.local:open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/default__vector_store.json
open file: /r

In [9]:
%%time

def _data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document
    
news_data=get_news_data("https://www.guancha.cn/internation/2024_06_20_738635.shtml")

document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
DEBUG:urllib3.connectionpool:https://www.guancha.cn:443 "GET /internation/2024_06_20_738635.shtml HTTP/1.1" 200 15661
https://www.guancha.cn:443 "GET /internation/2024_06_20_738635.shtml HTTP/1.1" 200 15661
https://www.guancha.cn:443 "GET /internation/2024_06_20_738635.shtml HTTP/1.1" 200 15661
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 【文/观察者网 齐倩】6月，沙特进入一年中最热月份，高温导致正在“圣城”麦加参加朝觐活动的大批...
> Adding chunk: 【文/观察者网 齐倩】6月，沙特进入一年中最热月份，高温导致正在“圣城”麦加参加朝觐活动的大批...
> Adding chunk: 【文/观察者网 齐倩】6月，沙特进入一年中最热月份，高温导致正在“圣城”麦加参加朝觐活动的大批...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 由于缺乏官方数据，死亡人数尚未确定，且可能会继续上升。
美联社19日报道称，网上流传的一份名单...
> Adding chunk: 由于缺乏官方数据，死亡人数尚未确定，且可能会继续上升。
美联社19日报道称，网上流传的一份名单...
> Adding chunk: 由于缺乏官方数据，死亡人数尚未确定，且可能会继续上升。
美联社19日报道称，网上流传的一份名单...
DEBUG:llama_index.c

In [10]:
%%time

doc_summary_index=_load_index('index')

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=1
)

response = query_engine.query("麦加参加朝觐活动发生了啥？")
response.print_response_stream()

DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from index/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from index/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from index/docstore.json.
DEBUG:fsspec.local:open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/docstore.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/docstore.json
open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/docstore.json
DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from index/index_store.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from index/index_store.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from index/index_store.json.
DEBUG:fsspec.local:open file: /root/notebook/my-jupyter-notebook/llm/llama_index/agent/index/index_store.jso

In [11]:
%%time

response = query_engine.query("清洁能源给“看不见的50亿人”带来了什么？")
response.print_response_stream()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
DEBUG:urllib3.connectionpool:http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': "Context information from multi

In [12]:
%%time

response = query_engine.query("堂下何人竟敢状告本官？")
response.print_response_stream()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
DEBUG:urllib3.connectionpool:http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': "Context information from multi

In [13]:
news_data=get_news_data("https://www.guancha.cn/politics/2024_06_20_738662.shtml")

document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
DEBUG:urllib3.connectionpool:https://www.guancha.cn:443 "GET /politics/2024_06_20_738662.shtml HTTP/1.1" 200 13375
https://www.guancha.cn:443 "GET /politics/2024_06_20_738662.shtml HTTP/1.1" 200 13375
https://www.guancha.cn:443 "GET /politics/2024_06_20_738662.shtml HTTP/1.1" 200 13375
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 据横琴粤澳深度合作区网站6月20日消息，横琴粤澳深度合作区执行委员会印发《关于促进横琴粤澳深度...
> Adding chunk: 据横琴粤澳深度合作区网站6月20日消息，横琴粤澳深度合作区执行委员会印发《关于促进横琴粤澳深度...
> Adding chunk: 据横琴粤澳深度合作区网站6月20日消息，横琴粤澳深度合作区执行委员会印发《关于促进横琴粤澳深度...
current doc id: 横琴粤澳深度合作区：全面取消商品住房限购
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answ

In [14]:
news_data=get_news_data("https://www.guancha.cn/internation/2024_06_20_738628.shtml")

document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
DEBUG:urllib3.connectionpool:https://www.guancha.cn:443 "GET /internation/2024_06_20_738628.shtml HTTP/1.1" 200 15948
https://www.guancha.cn:443 "GET /internation/2024_06_20_738628.shtml HTTP/1.1" 200 15948
https://www.guancha.cn:443 "GET /internation/2024_06_20_738628.shtml HTTP/1.1" 200 15948
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 【文/观察者网 熊超然】当地时间6月20日凌晨，俄罗斯总统普京抵达越南首都河内，开始对越南进行...
> Adding chunk: 【文/观察者网 熊超然】当地时间6月20日凌晨，俄罗斯总统普京抵达越南首都河内，开始对越南进行...
> Adding chunk: 【文/观察者网 熊超然】当地时间6月20日凌晨，俄罗斯总统普京抵达越南首都河内，开始对越南进行...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 去年，此类交易占双边贸易的40%以上，今年第一季度，这一比例上升至近60%。”他补充称：“这符...
> Adding chunk: 去年，此类交易占双边贸易的40%以上，今年第一季度，这一比例上升至近60%。”他补充称：“这符...
> Adding chunk: 去年，此类交易占双边贸易的40%以上，今年第一季度，这一比例上升至近60%。”他补充称：“这符...
DEBUG:llama_index.c

In [15]:
news_data=get_news_data("https://www.guancha.cn/GongSi/2024_06_20_738665.shtml")

document=_data2doc(news_data)
doc_summary_index.insert(document)
doc_summary_index.refresh([document])
doc_summary_index.storage_context.persist('index')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
Starting new HTTPS connection (1): www.guancha.cn:443
DEBUG:urllib3.connectionpool:https://www.guancha.cn:443 "GET /GongSi/2024_06_20_738665.shtml HTTP/1.1" 200 15270
https://www.guancha.cn:443 "GET /GongSi/2024_06_20_738665.shtml HTTP/1.1" 200 15270
https://www.guancha.cn:443 "GET /GongSi/2024_06_20_738665.shtml HTTP/1.1" 200 15270
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: （文/张志峰 编辑/马媛媛）母公司复星国际为债务问题寻求出售海南亚特兰蒂斯度假村的消息尚未尘埃...
> Adding chunk: （文/张志峰 编辑/马媛媛）母公司复星国际为债务问题寻求出售海南亚特兰蒂斯度假村的消息尚未尘埃...
> Adding chunk: （文/张志峰 编辑/马媛媛）母公司复星国际为债务问题寻求出售海南亚特兰蒂斯度假村的消息尚未尘埃...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 当之无愧地可以称为整个复星系最具标志性的大型文旅综合体项目。
因此，彼时关于复星系再度陷入债务...
> Adding chunk: 当之无愧地可以称为整个复星系最具标志性的大型文旅综合体项目。
因此，彼时关于复星系再度陷入债务...
> Adding chunk: 当之无愧地可以称为整个复星系最具标志性的大型文旅综合体项目。
因此，彼时关于复星系再度陷入债务...
current doc id: 旅游产业复苏，复星系豫园股份17.9

In [16]:
%%time

response = query_engine.query("清洁能源给“看不见的50亿人”带来了什么？")
response.print_response_stream()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
DEBUG:urllib3.connectionpool:http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': "Context information from multi

In [17]:
%%time

response = query_engine.query("沙特当局采取了哪些具体措施来确保安全并防止此类事故再次发生？")
response.print_response_stream()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
DEBUG:urllib3.connectionpool:http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': "Context information from multi

In [18]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=1,
)

retrieved_nodes = retriever.retrieve("沙特当局采取了哪些具体措施来确保安全并防止此类事故再次发生？")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
DEBUG:urllib3.connectionpool:http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 115 ms


In [19]:
len(retrieved_nodes)

1

In [22]:
retrieved_nodes[0]

NodeWithScore(node=TextNode(id_='5082f530-1c43-4c48-9ff2-ab25c1d32bc6', embedding=None, metadata={'title': '横琴粤澳深度合作区：全面取消商品住房限购', 'publish_time': '2024-06-20 17:43:01', 'author': '戴佳源', 'url': 'https://www.guancha.cn/politics/2024_06_20_738662.shtml', 'images': ['https://i.guancha.cn/news/official/2024/06/20/20240620173836682.png']}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='横琴粤澳深度合作区：全面取消商品住房限购', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '横琴粤澳深度合作区：全面取消商品住房限购', 'publish_time': '2024-06-20 17:43:01', 'author': '戴佳源', 'url': 'https://www.guancha.cn/politics/2024_06_20_738662.shtml', 'images': ['https://i.guancha.cn/news/official/2024/06/20/20240620173836682.png']}, hash='a369ab48bceb0d2c3cc6a466f664345c7653509eb7679f7a2b6eea587f64278f')}, text='据横琴粤澳深度合作区网站6月20日消息，横琴粤澳深度合作区执行委员会印发《关于促进横琴粤澳深度合作区房地产市场平稳健康发展的若干措施》的通知，宣布全面取消商品住房限购，在合作区范围内购房商品住房，不再审核购房人资格，同时取消商品住房限售年限。\n全文如下：\n关于促进横琴

In [23]:
%%time
# use retriever as part of a query engine
from llama_index.core.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("沙特当局采取了哪些具体措施来确保安全并防止此类事故再次发生？")
print(response)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
Starting new HTTP connection (1): 192.168.0.72:11435
DEBUG:urllib3.connectionpool:http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
http://192.168.0.72:11435 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': "Context information from multi